## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [30]:
f.predictions('ccl')

CCL 5m Interval Timestamp: 2024-10-18 03:18:08 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,up
kelly_1:2.5,1.0,0.319646,0.478296,0.351615
prob_up,0.0,0.484851,0.359531,0.536868
prob_static,1.0,0.514033,0.627354,0.440779
prob_down,0.0,0.001116,0.013115,0.022353
precision,0.75,0.738854,0.744136,0.668172
recall,0.746835,0.734177,0.736287,0.625793
f1,0.748414,0.736508,0.740191,0.646288
support,"[473.0, 474.0, 473.0]","[473.0, 474.0, 473.0]","[473.0, 474.0, 473.0]","[473.0, 474.0, 473.0]"


CCL 15m Interval Timestamp: 2024-10-18 03:18:16 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,up,up
kelly_1:2.5,0.125,0.653759,0.673842,0.462702
prob_up,0.375,0.752685,0.76703,0.616215
prob_static,0.375,0.244022,0.212007,0.264455
prob_down,0.25,0.003293,0.020963,0.11933
precision,0.657459,0.772059,0.796875,0.662338
recall,0.748428,0.65625,0.6375,0.6375
f1,0.7,0.709459,0.708333,0.649682
support,"[160.0, 159.0, 160.0]","[160.0, 159.0, 160.0]","[160.0, 159.0, 160.0]","[160.0, 159.0, 160.0]"


CCL 30m Interval Timestamp: 2024-10-18 03:18:19 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,static,up,up
kelly_1:2.5,0.3,0.268227,0.335537,0.222383
prob_up,0.125,0.463133,0.525383,0.444559
prob_static,0.375,0.477305,0.364296,0.406047
prob_down,0.5,0.059562,0.110321,0.149393
precision,0.777027,0.634483,0.742671,0.672598
recall,0.787671,0.632302,0.77551,0.642857
f1,0.782313,0.633391,0.758735,0.657391
support,"[294.0, 291.0, 292.0]","[294.0, 291.0, 292.0]","[294.0, 291.0, 292.0]","[294.0, 291.0, 292.0]"


CCL 1wk Interval Timestamp: 2024-10-18 03:18:37 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.65,0.98962,0.987479,0.84769
prob_up,0.75,0.992586,0.991056,0.891207
prob_static,0.25,0.005651,0.005785,0.064789
prob_down,0.0,0.001764,0.003159,0.044003
precision,0.738462,0.725806,0.71875,0.639344
recall,0.827586,0.775862,0.793103,0.672414
f1,0.780488,0.75,0.754098,0.655462
support,"[58.0, 56.0, 59.0]","[58.0, 56.0, 59.0]","[58.0, 56.0, 59.0]","[58.0, 56.0, 59.0]"


CCL 1mo Interval Timestamp: 2024-10-18 03:18:40 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.65,0.984554,0.884051,0.712469
prob_up,0.75,0.988967,0.917179,0.794621
prob_static,0.25,0.009281,0.062088,0.185991
prob_down,0.0,0.001752,0.020733,0.019389
precision,0.677419,0.782609,0.730769,0.62963
recall,0.875,0.75,0.791667,0.708333
f1,0.763636,0.765957,0.76,0.666667
support,"[24.0, 26.0, 24.0]","[24.0, 26.0, 24.0]","[24.0, 26.0, 24.0]","[24.0, 26.0, 24.0]"


### Check Summary Output

In [24]:
symbol='NVDA'
interval='5m'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

DL Time: 2024-10-17 15:45:25


,model,prediction,kelly_1:2.5,prob_up,prob_static,prob_down,precision,recall,f1,support
0,XGBoost,down,0.650000,0.125000,0.125000,0.750000,0.806723,0.763419,0.784474,"[504.0, 502.0, 503.0]"
1,GradientBoosting,down,0.676464,0.001311,0.229787,0.768903,0.790123,0.763419,0.776542,"[504.0, 502.0, 503.0]"
2,RandomForest,down,0.831881,0.013198,0.106887,0.879915,0.806005,0.693837,0.745726,"[504.0, 502.0, 503.0]"
3,KNN,down,0.388917,0.093694,0.342793,0.563512,0.717445,0.580517,0.641758,"[504.0, 502.0, 503.0]"


In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')